In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [2]:
class FederatedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.net(x)

In [3]:
def load_local_data(csv_path):
    df = pd.read_csv(csv_path)

    X = df[['traffic_load', 'num_users', 'packet_count', 'avg_packet_size']].values
    y = df['allocated_bandwidth'].values

    X = torch.tensor(X, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    dataset = TensorDataset(X, y)
    return DataLoader(dataset, batch_size=32, shuffle=True)

In [4]:
def local_train(model, dataloader, epochs=5):
    model.train()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for _ in range(epochs):
        for xb, yb in dataloader:
            optimizer.zero_grad()
            loss = criterion(model(xb), yb)
            loss.backward()
            optimizer.step()

    return model.state_dict()

In [5]:
def federated_average(weights_list):
    avg_weights = {}

    for key in weights_list[0].keys():
        avg_weights[key] = sum(w[key] for w in weights_list) / len(weights_list)

    return avg_weights

In [6]:
num_rounds = 10
local_epochs = 5

global_model = FederatedModel()

edge_files = [
    "../data/edge_node_1.csv",
    "../data/edge_node_2.csv",
    "../data/edge_node_3.csv"
]

for round_num in range(num_rounds):
    local_weights = []

    for edge_file in edge_files:
        local_model = FederatedModel()
        local_model.load_state_dict(global_model.state_dict())

        dataloader = load_local_data(edge_file)
        weights = local_train(local_model, dataloader, epochs=local_epochs)

        local_weights.append(weights)

    # Server aggregation
    global_weights = federated_average(local_weights)
    global_model.load_state_dict(global_weights)

    print(f"Federated Round {round_num+1} completed")

Federated Round 1 completed
Federated Round 2 completed
Federated Round 3 completed
Federated Round 4 completed
Federated Round 5 completed
Federated Round 6 completed
Federated Round 7 completed
Federated Round 8 completed
Federated Round 9 completed
Federated Round 10 completed


In [7]:
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/final_ran_dataset.csv")

X = df[['traffic_load', 'num_users', 'packet_count', 'avg_packet_size']].values
y = df['allocated_bandwidth'].values

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

global_model.eval()
with torch.no_grad():
    preds = global_model(X_test)
    mse = nn.MSELoss()(preds, y_test)

print("Federated Model Test MSE:", mse.item())

Federated Model Test MSE: 0.00013410938845481724


In [8]:
torch.save(global_model.state_dict(), "federated_global_model.pth")
print("Federated global model saved")

Federated global model saved
